In [61]:
import pandas as pd
from dataclasses import dataclass
from collections import Counter
import random
import math
import numpy as np
from icecream import ic
from matplotlib import pyplot as plt
from itertools import accumulate
from itertools import combinations
from tqdm.auto import tqdm
import geopy.distance
from geopy.distance import geodesic
import networkx as nx
from collections import deque

In [62]:
cities = pd.read_csv('cities/us.csv', header=None, names=['City', 'x', 'y'])
cities

,City,x,y
0,Abilene,32.454514,-99.738147
1,Akron,41.080456,-81.521429
2,Albuquerque,35.105552,-106.647388
3,Alexandria,38.818343,-77.082026
4,Allen,33.107224,-96.674676
...,...,...,...
321,Wichita Falls,33.906699,-98.525854
322,Wilmington,34.209225,-77.885767
323,Winston‐Salem,36.103262,-80.260578
324,Worcester,42.269478,-71.807783


In [63]:
DIST_MATRIX = np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.x, c1.y), (c2.x, c2.y)
    ).km

In [64]:
def fitness(solution):
    tot_dist=0
    for node in range(len(solution)-1):
        tot_dist -= DIST_MATRIX[solution[node], solution[node+1]]
    return tot_dist

In [65]:
def swap_mutation(solution):
    index = random.randint(1, len(solution)-3) #not the last one nor the first
    index2=index #should be higher
    while index2<=index:
        index2 = random.randint(1, len(solution)-2)
    selected_edge1 = solution[index]
    selected_edge2 = solution[index2]
    solution[index] = selected_edge2
    solution[index2] = selected_edge1
    return solution

In [66]:
@dataclass
class Individual:
    genome: list
    fitness : float = None

In [67]:
def find_closest(segments, city, visited):
    # Filtra i segmenti che contengono 'city' e hanno un'altra città non in 'visited'
    candidates = [
        (pair, distance) for pair, distance in segments
        if city in pair and (other_city := (pair - {city}).pop()) not in visited
    ]
    if len(candidates)==0:
        print("errore")
    closest_segment = min(candidates, key=lambda x: x[1])
    closest_city = (closest_segment[0] - {city}).pop()
    return (int(city), closest_city)

In [68]:
def greedy_sol(city, segments):
    solution = []
    solution.append(city)
    visited = []
    visited.append(int(city))
    while len(visited)<len(cities):
        _, c1 = find_closest(segments, city, visited)
        solution.append(c1)
        visited.append(c1)
        city=c1
    solution.append(solution[0])
    
    return solution

In [69]:
# def parent_selection(population):
#     candidates = sorted(np.random.choice(population, int(len(population)/8)), key=lambda e: e.fitness, reverse=True)
#     return candidates[0]


def inver_over_crossover(parent1, parent2):
    # Copia del primo genitore, su cui applicheremo le modifiche
    child = parent1[:]
    
    # Selezione di un punto di crossover casuale
    crossover_point = random.randint(1, len(parent1) - 2)
    
    # Segmento dal primo genitore (prima del punto di crossover)
    first_segment = parent1[:crossover_point]
    
    second_segment = []
    for city in parent2:
        if city not in first_segment:
            second_segment.append(city)
    
    # Combinazione dei segmenti per creare il figlio
    child = first_segment + second_segment
    
    return child

In [70]:
def inver_over_crossover(parent1, parent2):
    # Copia del primo genitore come base per il figlio
    child = parent1[:]
    
    # Mantieni il primo nodo identico all'ultimo
    start_node = child[0]
    end_node = child[-1]

    if start_node != end_node:
        child.append(start_node)
    
    # Iterazioni di inversioni casuali
    num_inversions = random.randint(1, len(parent1) // 2)  # Numero casuale di inversioni
    
    for _ in range(num_inversions):
        # Seleziona due punti casuali per definire l'intervallo da invertire
        i, j = sorted(random.sample(range(1, len(child) - 1), 2))
        
        # Inversione del sotto-percorso selezionato
        child[i:j + 1] = reversed(child[i:j + 1])
    
    # Assicurati che il figlio sia ciclico (chiudi il ciclo)
    if child[-1] != child[0]:
        child[-1] = child[0]
    
    return child

In [71]:
def create_random_solution():
    solution =[i for i in range(len(cities))]
    np.random.shuffle(solution)
    solution.append(solution[0])
    return solution

Actual algorithm

# Scramble mutation with strength parameter

In [72]:
def scramble_mutation(solution, strength = 0.4):
    # use the beta distribution to get a number n considering the strength:
    alpha = (1 - strength) * 5 + 1 
    beta = strength * 5 + 1
    # n between 1 and len(solution)-2
    max_n = len(solution) - 2
    n = int(1 + (max_n - 2) * random.betavariate(alpha, beta))
    indices = random.sample(range(1, len(solution)-3), n) #not the last one nor the first
    # shuffle the value of the indices:
    valori_da_mescolare = [solution[i] for i in indices]
    random.shuffle(valori_da_mescolare)
    # Riassegna i valori mescolati agli stessi indici in solution
    for i, indice in enumerate(indices):
        solution[indice] = valori_da_mescolare[i]
    return solution

### more population + selective pressure doesn't show better results: better configuration: population 20 and offspring 200.

In [73]:
def parent_selection(population):
    candidates = sorted(np.random.choice(population, int(len(population)/4)), key=lambda e: e.fitness, reverse=True)
    return candidates[0]

#### WE CAN SAY THAT SWAP MUTATION IS THE BEST MUTATION FOR THIS ALGORITHM. I avoid to show it, but using inversion mutation the results are slightly worst than swap muation.

Also applying the simulated annealing to the solution provided by EA does not show better results!

# Simulated annealing + EA

But what if we first apply the SA and then the EA?

In [18]:
def simulated_annealing(solution):
    temperatura_iniziale = 100
    tasso_riscaldamento = 1.02
    it=0

    #one out of five approach
    miglioramenti_recenti = deque(maxlen=5) 
    miglioramenti_richiesti = 1  

    #stopping criteria:
    miglioramenti_recenti_stop = deque(maxlen=1000)
    miglioramenti_recenti_stop.append(True)

    # Initial solution: greedy one!
    x_corrente = solution
    costo_corrente = fitness(x_corrente)
    best_cost = costo_corrente
    best_sol = x_corrente

    temperatura = temperatura_iniziale
    while it<1_000:
        it+=1
        # tweak the solution
        rn = random.random()
        first_time=True
        while rn > 0.8 or first_time:
            first_time=False
            x_nuovo = swap_mutation(x_corrente.copy())
            costo_nuovo = fitness(x_nuovo)
            rn = random.random()
        
        # variation of fitness by changing sign
        delta_costo = costo_nuovo*(-1) - costo_corrente*(-1) 
        #we are sure the solution after swap mutation is valid if the previous was it.
            
        if delta_costo < 0 or (random.random() < math.exp(-delta_costo / temperatura) and delta_costo!=0):
            x_corrente = x_nuovo
            costo_corrente = costo_nuovo
            miglioramenti_recenti.append(True)
            miglioramenti_recenti_stop.append(True)
            if costo_corrente*(-1) < best_cost*(-1):
                best_cost = costo_corrente
                best_sol = x_corrente
        else:
            miglioramenti_recenti.append(False)
            miglioramenti_recenti_stop.append(False)

        if miglioramenti_recenti.count(True) > miglioramenti_richiesti:
            temperatura *= tasso_riscaldamento  # more exploration
        if miglioramenti_recenti.count(True) < miglioramenti_richiesti:
            temperatura /= tasso_riscaldamento
        
        if miglioramenti_recenti_stop.count(True)==0: #stop condition
            break
    return best_sol

In [ ]:
POPULATION_SIZE = 20
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(cities)), 2)
]
population = [Individual(simulated_annealing(greedy_sol(random.randint(0, len(cities)-1), segments))) for _ in range(int(POPULATION_SIZE))]
# while len(population)<POPULATION_SIZE:
#     population.append(Individual(genome=simulated_annealing(create_random_solution())))
for i in population:
    i.fitness = fitness(i.genome)
population.sort(key=lambda i: i.fitness, reverse=True)
print("best solution before EA: ", population[0].fitness)
last_improvement=0
current_fitness=float('inf')*(-1)
OFFSPRING_SIZE = 200
MAX_GENERATIONS=1_000


for g in range(MAX_GENERATIONS):
    offspring = []
    
    for _ in range (OFFSPRING_SIZE):
        if np.random.random()<0.4:#mutation probability:
            p=parent_selection(population)
            o=swap_mutation(p.genome.copy())
        else:
            i1 = parent_selection(population)
            i2 = parent_selection(population)
            o = inver_over_crossover(i1.genome.copy(), i2.genome.copy())
        offspring.append(Individual(genome=o))
        
    for i in offspring:
        i.fitness = fitness(i.genome)
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    print("sol so far at generation: ", g, "is :",fitness(population[0].genome))
    if fitness(population[0].genome)>current_fitness:
        current_fitness = fitness(population[0].genome)
        last_improvement=g

population.sort(key=lambda i: i.fitness, reverse=True)
population = population[:POPULATION_SIZE]
print("best solution after EA: ", population[0].fitness)
print("final solution:", population[0])
print("last improvement:", last_improvement)

best solution before EA:  -46777.49306846578
sol so far at generation:  0 is : -46777.49306846578
sol so far at generation:  1 is : -46777.49306846578
sol so far at generation:  2 is : -46777.49306846578
sol so far at generation:  3 is : -46777.49306846578
sol so far at generation:  4 is : -46777.49306846578
sol so far at generation:  5 is : -46777.49306846578
sol so far at generation:  6 is : -46777.49306846578
sol so far at generation:  7 is : -46777.49306846578
sol so far at generation:  8 is : -46589.128567544205
sol so far at generation:  9 is : -46589.128567544205
sol so far at generation:  10 is : -46589.128567544205
sol so far at generation:  11 is : -46589.128567544205
sol so far at generation:  12 is : -46589.128567544205
sol so far at generation:  13 is : -46589.128567544205
sol so far at generation:  14 is : -46589.128567544205
sol so far at generation:  15 is : -46589.128567544205
sol so far at generation:  16 is : -46589.128567544205
sol so far at generation:  17 is : -46

KeyboardInterrupt: 

In this version, we will try to change the steasy state approach used: at each generation we keep track of a smaller set of the population, for instance the 20 fittest solution. These, and only these are the starting point of the new population (created using mutation and crossover until the new population has the same length as POPULATION_SIZE). We also adopt the modern approach in a steady state environment.

In [76]:
POPULATION_SIZE = 300
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(cities)), 2)
]
population = [Individual(simulated_annealing(greedy_sol(random.randint(0, len(cities)-1), segments))) for _ in range(int(POPULATION_SIZE))]
for i in population:
    i.fitness = fitness(i.genome)
population.sort(key=lambda i: i.fitness, reverse=True)
print("best solution before EA: ", population[0].fitness) # just to keep an eye on the first best solution.

# save the current best solution:
last_improvement=0
current_fitness=float('inf')*(-1)
ELITE_SIZE = 20
MAX_GENERATIONS=1_000

for g in range(MAX_GENERATIONS):
    population.sort(key=lambda i: i.fitness, reverse=True)
    offspring = population[:ELITE_SIZE]

    # Update the best individual if needed
    if fitness(offspring[0].genome) > current_fitness:
        best_solution = offspring[0].genome
        best_fitness = fitness(best_solution)
    
    while len(offspring) < POPULATION_SIZE:
        if np.random.random()<0.4:#mutation probability:
            p=parent_selection(population)
            o=swap_mutation(p.genome.copy())
        else:
            i1 = parent_selection(population)
            i2 = parent_selection(population)
            o = inver_over_crossover(i1.genome.copy(), i2.genome.copy())
        offspring.append(Individual(genome=o, fitness=fitness(o)))
        
    population=offspring
    population.sort(key=lambda i: i.fitness, reverse=True)
    if g % 20 == 0:
        print("sol so far at generation: ", g, "is :",fitness(population[0].genome))

population.sort(key=lambda i: i.fitness, reverse=True)
population = population[:POPULATION_SIZE]
print("best solution after EA: ", population[0].fitness)
print("final solution:", population[0])
print("last improvement:", last_improvement)

best solution before EA:  -45719.291345340294
sol so far at generation:  0 is : -45719.291345340294
sol so far at generation:  20 is : -45687.13856557104
sol so far at generation:  40 is : -45681.9646547668
sol so far at generation:  60 is : -45663.82923195641
sol so far at generation:  80 is : -45545.390541361485
sol so far at generation:  100 is : -45520.7201287253
sol so far at generation:  120 is : -45510.68709213746
sol so far at generation:  140 is : -45343.371811945806
sol so far at generation:  160 is : -45321.94441820172
sol so far at generation:  180 is : -45272.09847156529
sol so far at generation:  200 is : -45230.258338901054
sol so far at generation:  220 is : -45220.71122482118
sol so far at generation:  240 is : -45211.1021229449
sol so far at generation:  260 is : -45209.651760705645
sol so far at generation:  280 is : -45209.63235991636
sol so far at generation:  300 is : -45192.78120516433
sol so far at generation:  320 is : -45191.523706414126
sol so far at generati

KeyboardInterrupt: 